# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

Importar librerias y cargar los datos

In [1]:
import pandas as pd

# Cargar dataset
marketing_df = pd.read_csv(
    'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
)

# Ver las primeras filas
print(marketing_df.head())


   Unnamed: 0 Customer       State  Customer Lifetime Value Response  \
0           0  DK49336     Arizona              4809.216960       No   
1           1  KX64629  California              2228.525238       No   
2           2  LZ68649  Washington             14947.917300       No   
3           3  XL78013      Oregon             22332.439460      Yes   
4           4  QA50777      Oregon              9025.067525       No   

   Coverage Education Effective To Date EmploymentStatus Gender  ...  \
0     Basic   College           2/18/11         Employed      M  ...   
1     Basic   College           1/18/11       Unemployed      F  ...   
2     Basic  Bachelor           2/10/11         Employed      M  ...   
3  Extended   College           1/11/11         Employed      M  ...   
4   Premium  Bachelor           1/17/11    Medical Leave      F  ...   

   Number of Open Complaints Number of Policies     Policy Type        Policy  \
0                        0.0                  9  Corp

Revisar información general 

In [2]:
# Información general: columnas, tipos de datos y nulos
print(marketing_df.info())

# Estadísticas de columnas numéricas
print(marketing_df.describe())

# Revisar los nombres de las columnas
print(marketing_df.columns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10910 entries, 0 to 10909
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     10910 non-null  int64  
 1   Customer                       10910 non-null  object 
 2   State                          10279 non-null  object 
 3   Customer Lifetime Value        10910 non-null  float64
 4   Response                       10279 non-null  object 
 5   Coverage                       10910 non-null  object 
 6   Education                      10910 non-null  object 
 7   Effective To Date              10910 non-null  object 
 8   EmploymentStatus               10910 non-null  object 
 9   Gender                         10910 non-null  object 
 10  Income                         10910 non-null  int64  
 11  Location Code                  10910 non-null  object 
 12  Marital Status                 10910 non-null 

Limpieza de nombres de columnas y texto

In [3]:
# Nombres de columnas en minúsculas y sin espacios
marketing_df.columns = marketing_df.columns.str.lower().str.replace(' ', '_')

# Normalizar columnas de texto
text_cols = ['gender', 'education', 'policy_type', 'vehicle_class', 'sales_channel']
for col in text_cols:
    marketing_df[col] = marketing_df[col].str.strip().str.title()

# Eliminar duplicados
marketing_df.drop_duplicates(inplace=True)


1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

In [4]:
# Filtrar clientes
low_claim_yes = marketing_df[
    (marketing_df['total_claim_amount'] < 1000) &
    (marketing_df['response'].str.strip().str.title() == 'Yes')
]

# Mostrar primeras filas y número de clientes
print(low_claim_yes.head())
print("Número de clientes con bajo total_claim_amount y respuesta Yes:", low_claim_yes.shape[0])


    unnamed:_0 customer       state  customer_lifetime_value response  \
3            3  XL78013      Oregon             22332.439460      Yes   
8            8  FM55990  California              5989.773931      Yes   
15          15  CW49887  California              4626.801093      Yes   
19          19  NJ54277  California              3746.751625      Yes   
27          27  MQ68407      Oregon              4376.363592      Yes   

    coverage education effective_to_date employmentstatus gender  ...  \
3   Extended   College           1/11/11         Employed      M  ...   
8    Premium   College           1/19/11         Employed      M  ...   
15     Basic    Master           1/16/11         Employed      F  ...   
19  Extended   College           2/26/11         Employed      F  ...   
27   Premium  Bachelor           2/28/11         Employed      F  ...   

    number_of_open_complaints number_of_policies     policy_type  \
3                         0.0                  2  Corp

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

In [5]:
# Filtrar clientes con respuesta Yes
yes_responders = marketing_df[marketing_df['response'].str.strip().str.title() == 'Yes']

# Pivot table: promedio de monthly_premium_auto y customer_lifetime_value
pivot_profit = yes_responders.pivot_table(
    index='policy_type',
    columns='gender',
    values=['monthly_premium_auto', 'customer_lifetime_value'],
    aggfunc='mean'
).round(2)

print(pivot_profit)


               customer_lifetime_value          monthly_premium_auto       
gender                               F        M                    F      M
policy_type                                                                
Corporate Auto                 7712.63  7944.47                94.30  92.19
Personal Auto                  8339.79  7448.38                99.00  91.09
Special Auto                   7691.58  8247.09                92.31  86.34


3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [6]:
# Contar clientes por estado
customers_per_state = marketing_df.groupby('state')['customer'].count().reset_index(name='total_customers')

# Filtrar estados con más de 500 clientes
states_large = customers_per_state[customers_per_state['total_customers'] > 500].sort_values(by='total_customers', ascending=False)

print(states_large)


        state  total_customers
1  California             3552
3      Oregon             2909
0     Arizona             1937
2      Nevada              993
4  Washington              888


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [7]:
# Pivot table con estadísticas
clv_stats = marketing_df.pivot_table(
    index='education',
    columns='gender',
    values='customer_lifetime_value',
    aggfunc=['max', 'min', 'median']
).round(2)

print(clv_stats)


                           max                min            median         
gender                       F         M        F        M        F        M
education                                                                   
Bachelor              73225.96  67907.27  1904.00  1898.01  5640.51  5548.03
College               61850.19  61134.68  1898.68  1918.12  5623.61  6005.85
Doctor                44856.11  32677.34  2395.57  2267.60  5332.46  5577.67
High School Or Below  55277.45  83325.38  2144.92  1940.98  6039.55  6286.73
Master                51016.07  50568.26  2417.78  2272.31  5729.86  5579.10


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here